ライブラリインポート

In [2]:
import pandas as pd
import numpy as np

#import pycaret

#setup関数をimport
#from pycaret.regression import setup

#compare_models関数は、ライブラリ内のすべてのモデルを使って訓練を行い、スコアを評価する。
#回帰問題なので'refression'を指定
#from pycaret.regression import compare_models

#models関数により、すべての使用可能な機械学習モデルを確認できる。
#from pycaret.regression import models

#ceate_model関数:「交差検証」を用いて個別のモデルの訓練と評価を行う。  
#from pycaret.regression import create_model

#tune_model関数を用いて、ハイパーパラメータを最適化する。
#from pycaret.regression import tune_model

#plot_model関数を使い、各特徴量の重要度をプロットする、
#from pycaret.regression import plot_model

#finalize_model関数により、すべての訓練データを使ってモデルを訓練し、本番用のモデルを作成できるようにする。
#from pycaret.regression import finalize_model

#from pycaret.regression import predict_model

import lightgbm as lgb
from sklearn import metrics
from scipy.stats import rankdata
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import warnings
warnings.simplefilter('ignore', FutureWarning)

import re
import pickle
import seaborn as sns
import lightgbm as lgb
#import japanize_matplotlib
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

import copy

import statsmodels.api as sm

# 検証６

lightgbm(simple)

In [3]:
#前処理

#trainデータ処理
train_df = pd.read_table('train.tsv')
#One-Hot Encoding
train_df_one_hot_encoded = pd.get_dummies(train_df,columns=['weathersit'])
#曜日の'weekday'を土日(0)と平日(1)の2値データする。
#新しい変数'weekday2'を作って、全て0(zero)を入れる
train_df_one_hot_encoded['weekday2'] = 0
#平日を1にしたいので、1にするコードを用意する
train_df_one_hot_encoded.loc[train_df_one_hot_encoded['weekday'].isin(list(range(1,6))),'weekday2']=1
# 結果の件数を確認する
#train_df_one_hot_encoded['weekday2'].value_counts()
# id2500以前は、傾向が違うため、カット(運用開始など？)
train_df_one_hot_encoded = train_df_one_hot_encoded[train_df_one_hot_encoded['id'] > 2500]


#testデータ処理
test_df = pd.read_table('test.tsv')
#One-Hot Encoding
test_df_one_hot_encoded = pd.get_dummies(test_df,columns=['weathersit'])
#曜日の'weekday'を土日(0)と平日(1)の2値データする。
#新しい変数'weekday2'を作って、全て0(zero)を入れる
test_df_one_hot_encoded['weekday2'] = 0
#平日を1にしたいので、1にするコードを用意する
test_df_one_hot_encoded.loc[test_df_one_hot_encoded['weekday'].isin(list(range(1,6))),'weekday2']=1


In [5]:
train_df_one_hot_encoded.head()

,id,dteday,season,yr,mnth,hr,holiday,weekday,workingday,temp,atemp,hum,windspeed,cnt,weathersit_1,weathersit_2,weathersit_3,weathersit_4,weekday2
2500,2501,2011-04-19,2,0,4,2,0,2,1,0.46,0.4545,0.67,0.2836,13,1,0,0,0,1
2501,2502,2011-04-19,2,0,4,3,0,2,1,0.48,0.4697,0.63,0.2239,3,1,0,0,0,1
2502,2503,2011-04-19,2,0,4,4,0,2,1,0.46,0.4545,0.67,0.0896,5,1,0,0,0,1
2503,2504,2011-04-19,2,0,4,5,0,2,1,0.46,0.4545,0.72,0.1940,18,0,1,0,0,1
2504,2505,2011-04-19,2,0,4,6,0,2,1,0.48,0.4697,0.67,0.0000,65,0,1,0,0,1


In [6]:
#必要な説明変数だけをとりだす
train_df_x = train_df_one_hot_encoded[['hr','workingday','weekday2','temp','atemp','weathersit_1','weathersit_2','weathersit_3']]
train_df_y = train_df_one_hot_encoded['cnt']
train_df_x.head()

,hr,workingday,weekday2,temp,atemp,weathersit_1,weathersit_2,weathersit_3
2500,2,1,1,0.46,0.4545,1,0,0
2501,3,1,1,0.48,0.4697,1,0,0
2502,4,1,1,0.46,0.4545,1,0,0
2503,5,1,1,0.46,0.4545,0,1,0
2504,6,1,1,0.48,0.4697,0,1,0


In [7]:
# defaultはtrain dataが75%、test dataが25%に分割
# また時系列のデータなので、shuffleをFalseにする。
train_X, test_X, train_y, test_y = train_test_split(train_df_x, train_df_y, random_state=42, shuffle=False)


In [8]:
import lightgbm as lgb #LightGBM

In [10]:
# モデルのインスタンス
model_lgb = lgb.LGBMRegressor()

In [11]:
# モデルの学習
model_lgb.fit(train_X, train_y)

LGBMRegressor()

In [14]:
pred_lgb = model_lgb.predict(test_X)

In [15]:
pred_lgb

array([227.55283571,  50.30834641,   9.07459526, ..., 134.80874393,
        95.38747537,  65.97230913])

In [13]:
# 評価(平均２乗誤差)
from sklearn.metrics import mean_squared_error

In [16]:
# 訓練データ予測のRMSE計算
rmse = np.sqrt(mean_squared_error(test_y, pred_lgb))
print('RMSE : {:.3f}'.format(rmse))


RMSE : 58.273


lightgbmだけでかなり良い感じ

未知のtestデータを予測する。

In [17]:
#testデータの必要な変数だけをとりだす
test_df_x = test_df_one_hot_encoded[['hr','workingday','weekday2','temp','atemp','weathersit_1','weathersit_2','weathersit_3']]
test_df_x.head()

,hr,workingday,weekday2,temp,atemp,weathersit_1,weathersit_2,weathersit_3
0,0,0,0,0.36,0.3788,1,0,0
1,1,0,0,0.36,0.3485,1,0,0
2,2,0,0,0.32,0.3485,1,0,0
3,3,0,0,0.30,0.3333,1,0,0
4,4,0,0,0.28,0.3030,1,0,0


In [18]:
test_df_x.shape

(8734, 8)

In [21]:
# test dataにモデルを適用する
Y_test_pred = model_lgb.predict(test_df_x) #X_testデータを使って予測する

In [22]:
Y_test_pred

array([ 95.37698111,  62.87664809,  37.81264076, ..., 104.15462424,
        73.15663717,  41.19819616])

### 提出ファイルの作成

In [23]:
Y_test_pred.shape,test_df.shape

((8734,), (8734, 14))

In [24]:
sub = test_df.copy()
sub['cnt'] = Y_test_pred
sub.head(5)

,id,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,8646,2012-01-01,1,1,1,0,0,0,0,1,0.36,0.3788,0.66,0.0000,95.376981
1,8647,2012-01-01,1,1,1,1,0,0,0,1,0.36,0.3485,0.66,0.1343,62.876648
2,8648,2012-01-01,1,1,1,2,0,0,0,1,0.32,0.3485,0.76,0.0000,37.812641
3,8649,2012-01-01,1,1,1,3,0,0,0,1,0.30,0.3333,0.81,0.0000,18.378247
4,8650,2012-01-01,1,1,1,4,0,0,0,1,0.28,0.3030,0.81,0.0896,3.826015


In [25]:
# 提出ファイル
sub[['id','cnt']].to_csv('sample_submit_06.csv', index=False, header = False)

In [26]:
sample_submit_06 = pd.read_csv('sample_submit_06.csv')
sample_submit_06.head(5)

,8646,95.37698110542098
0,8647,62.876648
1,8648,37.812641
2,8649,18.378247
3,8650,3.826015
4,8651,5.940500


### 評価  
RMSE  115.5676928(125/1246位)  
精度は高いとは言えない、  
XGboostの方が強い。

# 検証7

## ランダムフォレスト

In [27]:
from sklearn.ensemble import RandomForestRegressor #ランダムフォレスト 

In [28]:
#前処理

#trainデータ処理
train_df = pd.read_table('train.tsv')
#One-Hot Encoding
train_df_one_hot_encoded = pd.get_dummies(train_df,columns=['weathersit'])
#曜日の'weekday'を土日(0)と平日(1)の2値データする。
#新しい変数'weekday2'を作って、全て0(zero)を入れる
train_df_one_hot_encoded['weekday2'] = 0
#平日を1にしたいので、1にするコードを用意する
train_df_one_hot_encoded.loc[train_df_one_hot_encoded['weekday'].isin(list(range(1,6))),'weekday2']=1
# 結果の件数を確認する
#train_df_one_hot_encoded['weekday2'].value_counts()
# id2500以前は、傾向が違うため、カット(運用開始など？)
train_df_one_hot_encoded = train_df_one_hot_encoded[train_df_one_hot_encoded['id'] > 2500]


#testデータ処理
test_df = pd.read_table('test.tsv')
#One-Hot Encoding
test_df_one_hot_encoded = pd.get_dummies(test_df,columns=['weathersit'])
#曜日の'weekday'を土日(0)と平日(1)の2値データする。
#新しい変数'weekday2'を作って、全て0(zero)を入れる
test_df_one_hot_encoded['weekday2'] = 0
#平日を1にしたいので、1にするコードを用意する
test_df_one_hot_encoded.loc[test_df_one_hot_encoded['weekday'].isin(list(range(1,6))),'weekday2']=1


In [29]:
#必要な説明変数だけをとりだす
train_df_x = train_df_one_hot_encoded[['hr','workingday','weekday2','temp','atemp','weathersit_1','weathersit_2','weathersit_3']]
train_df_y = train_df_one_hot_encoded['cnt']
train_df_x.head()

,hr,workingday,weekday2,temp,atemp,weathersit_1,weathersit_2,weathersit_3
2500,2,1,1,0.46,0.4545,1,0,0
2501,3,1,1,0.48,0.4697,1,0,0
2502,4,1,1,0.46,0.4545,1,0,0
2503,5,1,1,0.46,0.4545,0,1,0
2504,6,1,1,0.48,0.4697,0,1,0


In [30]:
# defaultはtrain dataが75%、test dataが25%に分割
# また時系列のデータなので、shuffleをFalseにする。
train_X, test_X, train_y, test_y = train_test_split(train_df_x, train_df_y, random_state=42, shuffle=False)


In [31]:
# モデルのインスタンス
model_RF = RandomForestRegressor()

In [32]:
# モデルの学習
model_RF.fit(train_X, train_y)

RandomForestRegressor()

In [33]:
pred_RF = model_RF.predict(test_X)

In [34]:
pred_RF

array([176.50566667,  74.10666667,  46.70833333, ..., 146.85228571,
        99.33      ,  64.45333333])

In [35]:
# 訓練データ予測のRMSE計算
rmse = np.sqrt(mean_squared_error(test_y, pred_RF))
print('RMSE : {:.3f}'.format(rmse))


RMSE : 63.263


lightgbmよりは若干劣る

未知のtestデータを予測する。

In [36]:
#testデータの必要な変数だけをとりだす
test_df_x = test_df_one_hot_encoded[['hr','workingday','weekday2','temp','atemp','weathersit_1','weathersit_2','weathersit_3']]
test_df_x.head()

,hr,workingday,weekday2,temp,atemp,weathersit_1,weathersit_2,weathersit_3
0,0,0,0,0.36,0.3788,1,0,0
1,1,0,0,0.36,0.3485,1,0,0
2,2,0,0,0.32,0.3485,1,0,0
3,3,0,0,0.30,0.3333,1,0,0
4,4,0,0,0.28,0.3030,1,0,0


In [37]:
test_df_x.shape

(8734, 8)

In [42]:
# test dataにモデルを適用する
Y_test_pred = model_RF.predict(test_df_x) #X_testデータを使って予測する

In [43]:
Y_test_pred

array([ 85.485     ,  59.87333333,  37.72785714, ..., 120.835     ,
       105.403     ,  61.53      ])

### 提出ファイルの作成

In [44]:
Y_test_pred.shape,test_df.shape

((8734,), (8734, 14))

In [45]:
sub = test_df.copy()
sub['cnt'] = Y_test_pred
sub.head(5)

,id,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,8646,2012-01-01,1,1,1,0,0,0,0,1,0.36,0.3788,0.66,0.0000,85.485000
1,8647,2012-01-01,1,1,1,1,0,0,0,1,0.36,0.3485,0.66,0.1343,59.873333
2,8648,2012-01-01,1,1,1,2,0,0,0,1,0.32,0.3485,0.76,0.0000,37.727857
3,8649,2012-01-01,1,1,1,3,0,0,0,1,0.30,0.3333,0.81,0.0000,16.238000
4,8650,2012-01-01,1,1,1,4,0,0,0,1,0.28,0.3030,0.81,0.0896,6.460000


In [46]:
# 提出ファイル
sub[['id','cnt']].to_csv('sample_submit_07.csv', index=False, header = False)

In [47]:
sample_submit_07 = pd.read_csv('sample_submit_07.csv')
sample_submit_07.head(5)

,8646,85.485
0,8647,59.873333
1,8648,37.727857
2,8649,16.238000
3,8650,6.460000
4,8651,6.070000


### 評価  
RMSE  ?(125/1246位)  
精度は高いとは言えない、  
特徴量を加えたり、別の学習モデルやアンサンブル学習などいろいろ出来そう。

# 検証8

## 重回帰分析(前処理あり)

In [48]:
from sklearn.linear_model import LinearRegression #重回帰分析

In [49]:
#前処理

#trainデータ処理
train_df = pd.read_table('train.tsv')
#One-Hot Encoding
train_df_one_hot_encoded = pd.get_dummies(train_df,columns=['weathersit'])
#曜日の'weekday'を土日(0)と平日(1)の2値データする。
#新しい変数'weekday2'を作って、全て0(zero)を入れる
train_df_one_hot_encoded['weekday2'] = 0
#平日を1にしたいので、1にするコードを用意する
train_df_one_hot_encoded.loc[train_df_one_hot_encoded['weekday'].isin(list(range(1,6))),'weekday2']=1
# 結果の件数を確認する
#train_df_one_hot_encoded['weekday2'].value_counts()
# id2500以前は、傾向が違うため、カット(運用開始など？)
train_df_one_hot_encoded = train_df_one_hot_encoded[train_df_one_hot_encoded['id'] > 2500]


#testデータ処理
test_df = pd.read_table('test.tsv')
#One-Hot Encoding
test_df_one_hot_encoded = pd.get_dummies(test_df,columns=['weathersit'])
#曜日の'weekday'を土日(0)と平日(1)の2値データする。
#新しい変数'weekday2'を作って、全て0(zero)を入れる
test_df_one_hot_encoded['weekday2'] = 0
#平日を1にしたいので、1にするコードを用意する
test_df_one_hot_encoded.loc[test_df_one_hot_encoded['weekday'].isin(list(range(1,6))),'weekday2']=1


In [50]:
#必要な説明変数だけをとりだす
train_df_x = train_df_one_hot_encoded[['hr','workingday','weekday2','temp','atemp','weathersit_1','weathersit_2','weathersit_3']]
train_df_y = train_df_one_hot_encoded['cnt']
train_df_x.head()

,hr,workingday,weekday2,temp,atemp,weathersit_1,weathersit_2,weathersit_3
2500,2,1,1,0.46,0.4545,1,0,0
2501,3,1,1,0.48,0.4697,1,0,0
2502,4,1,1,0.46,0.4545,1,0,0
2503,5,1,1,0.46,0.4545,0,1,0
2504,6,1,1,0.48,0.4697,0,1,0


In [51]:
# defaultはtrain dataが75%、test dataが25%に分割
# また時系列のデータなので、shuffleをFalseにする。
train_X, test_X, train_y, test_y = train_test_split(train_df_x, train_df_y, random_state=42, shuffle=False)


In [52]:
# モデルのインスタンス
model_LR = LinearRegression()

In [53]:
# モデルの学習
model_LR.fit(train_X, train_y)

LinearRegression()

In [54]:
pred_LR = model_LR.predict(test_X)

In [55]:
pred_LR

array([190.86700578, 149.57834743, 153.25590258, ..., 234.0675898 ,
       238.99319003, 243.91057944])

In [56]:
# 訓練データ予測のRMSE計算
rmse = np.sqrt(mean_squared_error(test_y, pred_LR))
print('RMSE : {:.3f}'.format(rmse))


RMSE : 106.935


lightgbm,ランダムフォレストよりは若干劣る

未知のtestデータを予測する。

In [57]:
#testデータの必要な変数だけをとりだす
test_df_x = test_df_one_hot_encoded[['hr','workingday','weekday2','temp','atemp','weathersit_1','weathersit_2','weathersit_3']]
test_df_x.head()

,hr,workingday,weekday2,temp,atemp,weathersit_1,weathersit_2,weathersit_3
0,0,0,0,0.36,0.3788,1,0,0
1,1,0,0,0.36,0.3485,1,0,0
2,2,0,0,0.32,0.3485,1,0,0
3,3,0,0,0.30,0.3333,1,0,0
4,4,0,0,0.28,0.3030,1,0,0


In [58]:
test_df_x.shape

(8734, 8)

In [59]:
# test dataにモデルを適用する
Y_test_pred = model_LR.predict(test_df_x) #X_testデータを使って予測する

In [60]:
Y_test_pred

array([ 17.62665205,  29.95296302,  27.46964169, ..., 191.71440562,
       200.31300298, 210.15143461])

### 提出ファイルの作成

In [61]:
Y_test_pred.shape,test_df.shape

((8734,), (8734, 14))

In [62]:
sub = test_df.copy()
sub['cnt'] = Y_test_pred
sub.head(5)

,id,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,8646,2012-01-01,1,1,1,0,0,0,0,1,0.36,0.3788,0.66,0.0000,17.626652
1,8647,2012-01-01,1,1,1,1,0,0,0,1,0.36,0.3485,0.66,0.1343,29.952963
2,8648,2012-01-01,1,1,1,2,0,0,0,1,0.32,0.3485,0.76,0.0000,27.469642
3,8649,2012-01-01,1,1,1,3,0,0,0,1,0.30,0.3333,0.81,0.0000,32.395242
4,8650,2012-01-01,1,1,1,4,0,0,0,1,0.28,0.3030,0.81,0.0896,38.560676


In [63]:
# 提出ファイル
sub[['id','cnt']].to_csv('sample_submit_08.csv', index=False, header = False)

In [65]:
sample_submit_08 = pd.read_csv('sample_submit_08.csv')
sample_submit_08.head()

,8646,17.62665205468295
0,8647,29.952963
1,8648,27.469642
2,8649,32.395242
3,8650,38.560676
4,8651,49.638942


### 評価  
RMSE  ?(125/1246位)  
精度は高いとは言えない、  
特徴量を加えたり、別の学習モデルやアンサンブル学習などいろいろ出来そう。

# 検証9

## スタッキング(重回帰分析,ランダムフォレスト,lightgbmの予測値を重回帰分析にかけて最終的な予測をする。)

In [88]:
#前処理

#trainデータ処理
train_df = pd.read_table('train.tsv')
#One-Hot Encoding
train_df_one_hot_encoded = pd.get_dummies(train_df,columns=['weathersit'])
#曜日の'weekday'を土日(0)と平日(1)の2値データする。
#新しい変数'weekday2'を作って、全て0(zero)を入れる
train_df_one_hot_encoded['weekday2'] = 0
#平日を1にしたいので、1にするコードを用意する
train_df_one_hot_encoded.loc[train_df_one_hot_encoded['weekday'].isin(list(range(1,6))),'weekday2']=1
# 結果の件数を確認する
#train_df_one_hot_encoded['weekday2'].value_counts()
# id2500以前は、傾向が違うため、カット(運用開始など？)
train_df_one_hot_encoded = train_df_one_hot_encoded[train_df_one_hot_encoded['id'] > 2500]


#testデータ処理
test_df = pd.read_table('test.tsv')
#One-Hot Encoding
test_df_one_hot_encoded = pd.get_dummies(test_df,columns=['weathersit'])
#曜日の'weekday'を土日(0)と平日(1)の2値データする。
#新しい変数'weekday2'を作って、全て0(zero)を入れる
test_df_one_hot_encoded['weekday2'] = 0
#平日を1にしたいので、1にするコードを用意する
test_df_one_hot_encoded.loc[test_df_one_hot_encoded['weekday'].isin(list(range(1,6))),'weekday2']=1


In [89]:
#必要な説明変数だけをとりだす
train_df_x = train_df_one_hot_encoded[['hr','workingday','weekday2','temp','atemp','weathersit_1','weathersit_2','weathersit_3']]
train_df_y = train_df_one_hot_encoded['cnt']
train_df_x.head()

,hr,workingday,weekday2,temp,atemp,weathersit_1,weathersit_2,weathersit_3
2500,2,1,1,0.46,0.4545,1,0,0
2501,3,1,1,0.48,0.4697,1,0,0
2502,4,1,1,0.46,0.4545,1,0,0
2503,5,1,1,0.46,0.4545,0,1,0
2504,6,1,1,0.48,0.4697,0,1,0


第一段階での予測値を利用して、さらに重回帰分析をするので、データを三分割する必要がある。

In [90]:
# データを3つに分割
X_train,X_test, y_train, y_test = train_test_split(train_df_x, train_df_y, test_size=0.2, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)
print(y_train.shape)
print(y_valid.shape)
print(y_test.shape)

(3932, 8)
(984, 8)
(1229, 8)
(3932,)
(984,)
(1229,)


In [91]:
# モデルのインスタンス
model_1 = LinearRegression()
model_2 = RandomForestRegressor()
model_3 = lgb.LGBMRegressor()

# モデルの学習
model_1.fit(X_train, y_train)
model_2.fit(X_train, y_train)
model_3.fit(X_train, y_train)

# 予測値の作成
pred_1 = model_1.predict(X_test)
pred_2 = model_2.predict(X_test)
pred_3 = model_3.predict(X_test)


In [92]:
#　各モデル個別の予測精度をRMSEで確認
rmse_1 = np.sqrt(mean_squared_error(y_test, pred_1))
print('重回帰分析のRMSE : {:.3f}'.format(rmse_1))

rmse_2 = np.sqrt(mean_squared_error(y_test, pred_2))
print('ランダムフォレストのRMSE : {:.3f}'.format(rmse_2))

rmse_3 = np.sqrt(mean_squared_error(y_test, pred_3))
print('LightGBMのRMSE : {:.3f}'.format(rmse_3))


重回帰分析のRMSE : 122.550
ランダムフォレストのRMSE : 49.401
LightGBMのRMSE : 43.742


In [93]:
# 第一段階の予測値
first_pred_1 = model_1.predict(X_valid)
first_pred_2 = model_2.predict(X_valid)
first_pred_3 = model_3.predict(X_valid)

# 第一段階の予測値をまとめる（メタモデルの特徴量）
stack_pred = np.column_stack((first_pred_1,first_pred_2,first_pred_3))

# メタモデルの作成
meta_model = LinearRegression()
# 第一段階の予測値の答え = y_valid
meta_model.fit(stack_pred, y_valid)

# 事前に予測しておいた値でスタッキングの精度を確認する
stack_test_pred = np.column_stack((pred_1, pred_2, pred_3))
meta_test_pred = meta_model.predict(stack_test_pred)


In [94]:
#　メタモデルの予測精度をRMSEで確認
rmse_4 = np.sqrt(mean_squared_error(y_test, meta_test_pred))
print('メタモデルのRMSE : {:.3f}'.format(rmse_4))


メタモデルのRMSE : 43.829


LightGBM単体のRMSEの方が若干精度は良かった、、

未知のtestデータを予測する。

In [95]:
#testデータの必要な変数だけをとりだす
test_df_x = test_df_one_hot_encoded[['hr','workingday','weekday2','temp','atemp','weathersit_1','weathersit_2','weathersit_3']]
test_df_x.head()

,hr,workingday,weekday2,temp,atemp,weathersit_1,weathersit_2,weathersit_3
0,0,0,0,0.36,0.3788,1,0,0
1,1,0,0,0.36,0.3485,1,0,0
2,2,0,0,0.32,0.3485,1,0,0
3,3,0,0,0.30,0.3333,1,0,0
4,4,0,0,0.28,0.3030,1,0,0


In [96]:
test_df_x.shape

(8734, 8)

In [97]:
# 未知データの第一予測値の作成
pred_1 = model_1.predict(test_df_x)
pred_2 = model_2.predict(test_df_x)
pred_3 = model_3.predict(test_df_x)


In [98]:
pred_1

array([ 34.09714076,  44.48768322,  42.77253212, ..., 191.15220733,
       199.34632481, 208.2709713 ])

In [99]:

# 事前に予測しておいた値でスタッキングの精度を確認する
stack_test_pred = np.column_stack((pred_1, pred_2, pred_3))
meta_test_pred = meta_model.predict(stack_test_pred)


In [100]:
meta_test_pred

array([74.9579971 , 54.04241473, 41.73462372, ..., 98.5883789 ,
       80.32123493, 55.57004104])

### 提出ファイルの作成

In [101]:
meta_test_pred.shape,test_df.shape

((8734,), (8734, 14))

In [102]:
sub = test_df.copy()
sub['cnt'] = meta_test_pred
sub.head(5)

,id,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,8646,2012-01-01,1,1,1,0,0,0,0,1,0.36,0.3788,0.66,0.0000,74.957997
1,8647,2012-01-01,1,1,1,1,0,0,0,1,0.36,0.3485,0.66,0.1343,54.042415
2,8648,2012-01-01,1,1,1,2,0,0,0,1,0.32,0.3485,0.76,0.0000,41.734624
3,8649,2012-01-01,1,1,1,3,0,0,0,1,0.30,0.3333,0.81,0.0000,22.737160
4,8650,2012-01-01,1,1,1,4,0,0,0,1,0.28,0.3030,0.81,0.0896,16.442751


In [103]:
# 提出ファイル
sub[['id','cnt']].to_csv('sample_submit_09.csv', index=False, header = False)

In [104]:
sample_submit_09 = pd.read_csv('sample_submit_09.csv')
sample_submit_09.head()

,8646,74.95799709831172
0,8647,54.042415
1,8648,41.734624
2,8649,22.737160
3,8650,16.442751
4,8651,17.205421


### 評価  
RMSE  118.8784134(125/1246位)  
精度は高いとは言えない、  
単純なXGboostの方が強かった。